<h1>First Part (1)</h1>
<h3>Import Pandas and scrape the tables from the Wikipedia site using the .read_html function</h3>

In [ ]:
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_neighbour_tables = pd.read_html(url)

<h3>Select the right table for the project and display the first 5 records</h3>

In [113]:
df_neighbours = df_neighbour_tables[0]

In [114]:
df_neighbours.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h3>Drop all rows with the Borough value as 'Not assigned'</h3>

In [115]:
df_neighbours = df_neighbours[df_neighbours.Borough != "Not assigned"]

In [116]:
df_neighbours.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h3>Reset the index of the dataframe as a new dataframe called df_neighbours1</h3>

In [118]:
# len(df_neighbours['Postal Code'].unique())
# df_neighbours1 = df_neighbours
df_neighbours1 = df_neighbours
df_neighbours1 = df_neighbours1.reset_index(drop=True)

<h3>Replace any 'Not assigned' Neighbourhood with it's Borough</h3>

In [119]:

for index, row in df_neighbours1.iterrows():
    if (row["Neighbourhood"] == "Not assigned"):
        row["Neighbourhood"] = row["Neighbourhood"].replace('Not assigned',row["Borough"])
        print (row["Neighbourhood"])
    # print(row["Borough"], row["Neighbourhood"])


<h3>Display the number of rows and columns using .shape</h3>

In [120]:
df_neighbours1.shape

(103, 3)